In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

In [2]:
def run(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'
    }

    response = None
    for i in range(5):
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            break
        else:
            print('Failed attempt', i)
            time.sleep(2)

    if not response or response.status_code != 200:
        print('Failed to retrieve the webpage.')
        return

    soup = BeautifulSoup(response.text, 'lxml')

    reviews = soup.find_all('div', class_='review-row')

    data = []
    for review in reviews:
        critic_name = review.find('a', href=re.compile('/critics/')).text.strip()
        critic_link = 'https://www.rottentomatoes.com' + review.find('a', href=re.compile('/critics/'))['href']

        platform = review.find('a', class_='publication').text.strip()
        review_date = review.find('span', {'data-qa': 'review-date'}).text.strip()

        rating_icon = review.find('score-icon-critic-deprecated', {'size': 'small'})
        rating = 'NA'
        if rating_icon:
            state = rating_icon.get('state', 'unknown')
            rating = 'fresh' if state.lower() == 'fresh' else 'rotten'

        text = review.find('p', class_='review-text').text.strip()

        data.append([critic_name, text, rating, review_date, platform])

    # Create a DataFrame from the data
    df = pd.DataFrame(data, columns=['Critic', 'Text', 'Rating', 'Review Date', 'Platform'])

    # Save the DataFrame to a CSV file
    df.to_csv('reviews.csv', encoding='utf-8', index=False)

In [3]:
if __name__ == "__main__":
    url = 'https://www.rottentomatoes.com/m/matrix_revolutions/reviews'
    run(url)